**классификатор текстов LSTM + Embeddings**

Евгений Борисов <esborisov@sevsu.ru>

## Данные

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()        
pd.options.display.max_colwidth = 200 

In [2]:
ff = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount','unk']

data = pd.concat([
    pd.read_csv('data/positive.csv.gz',sep=';',header=None),
    pd.read_csv('data/negative.csv.gz',sep=';',header=None),
])

data.columns = ff

data = data[['id','ttext', 'ttype']]

print( 'negative:', len(data.query('ttype==-1')), '\npositive:',len(data.query('ttype==1')) )

data.sample(10)

negative: 111923 
positive: 114911


,id,ttext,ttype
36654,413684628047020032,"RT @WvvvvvVWv: Сегодня город парализован , все колом :(",-1
114110,411221240595357697,"Ну а чего мало кому должны что ли))) @ ЦУМ ""Валентина"" http://t.co/tG7Q9xFjYS",1
4731,409750902942474241,"В школе дали пробное ГИА, посмотрела не чего не поняла... что делать? Безысходность:(",-1
24243,411993311508193280,"На Гарри Поттера подсела(( оторваться не могу. Каждую часть, как заново смотрю. О это прекрасное чувство\n✨",-1
10582,409289016019001345,"Люблю своих одноклассников , они у меня самые лучшие ! :*",1
60413,410279201636634624,"надпись в зоопарке: ""страусов не пугать - пол бетонный...""))))",1
112888,411212198002712577,"Мдааа..Тупили друг друга как могли,вхаха,я думаю,потому что я так считаю,потому что я так думаю)",1
815,409084131730272256,"да мой любимый Маршак зарелизился, хоть и с новым вокалистом (",-1
10958,409294304603934721,"RT @ehapocah: Наверное, стоит дать им минутку побыть вдвоём, перед операцией. *^_^* #TEAMFOLLOWBACK",1
17902,409375562679844864,"@karrimov кекс новому читателю,ведь сегодня-День кекса!!!))) http://t.co/9yG8qnfnEZ",1


### очистка данных

In [3]:
# import re

In [4]:
# # применяет список замен pat к строке s
# def replace_patterns(s,pat):
#     if len(pat)<1: return s
#     return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# # нормализация текста
# def string_normalizer(s):
#     pat = [
#       #[r'ё','е'] # замена ё для унификации
#       #,[r'</?[a-z]+>',' '] # удаляем xml
#       [r'[:;]-*[)D]',' радость ']
#       ,[r'\)\)\)*',' радость ']
#       ,[r'[:;]\*',' поцелуй ']
#       ,[r':\(',' печаль ']
#       ,[r'\(\(\(*',' печаль ']
#     ]
#     return replace_patterns(s,pat).strip()

In [5]:
# data['ttext_'] = data['ttext'].progress_apply(string_normalizer)

In [6]:
# data

In [7]:
import re
from nltk.tokenize import word_tokenize as nltk_tokenize_word

In [8]:
def tokenize(line): # разбиваем предложения на слова
    return    [ 
        t.lower() 
        for t in nltk_tokenize_word(line) 
        # if re.match(r'[а-я -]+',t.lower()) and len(t)>1 
        #      if not( (t.lower() in stopwords) or (len(t)<3) ) 
    ] 

In [9]:
data['ttext_'] = data['ttext'].progress_apply(tokenize)

  0%|          | 0/226834 [00:00<?, ?it/s]

In [10]:
data.sample(10)

,id,ttext,ttype,ttext_
110115,424813288472719360,сейчас со всей мочи завою с тоски..... никто не услышит... (((,-1,"[сейчас, со, всей, мочи, завою, с, тоски, ....., никто, не, услышит, ..., (, (, (]"
52868,410060677189881856,"сегодня писала объяснительную почему опоздала на второй урок ._.\nпервая моя ""объяснительная"" за всю мою учебную жизнь :D",1,"[сегодня, писала, объяснительную, почему, опоздала, на, второй, урок, ._, ., первая, моя, ``, объяснительная, '', за, всю, мою, учебную, жизнь, :, d]"
46952,414774551248392192,"RT @Lud_Esh: На суздальских выборах заранее подписали протоколы http://t.co/VI32ef8oEk Химичат, гады (( #Суздаль",-1,"[rt, @, lud_esh, :, на, суздальских, выборах, заранее, подписали, протоколы, http, :, //t.co/vi32ef8oek, химичат, ,, гады, (, (, #, суздаль]"
3847,409577105631244288,"10-минутный ролик в 360р грузится 20 минут :( Интересно, я когда-нибудь смогу нормально смотреть потоковое видео?",-1,"[10-минутный, ролик, в, 360р, грузится, 20, минут, :, (, интересно, ,, я, когда-нибудь, смогу, нормально, смотреть, потоковое, видео, ?]"
98434,423004458164162560,"Никому не нравится наш класс, мы бичи:(\nВсе идут к Бшкам.",-1,"[никому, не, нравится, наш, класс, ,, мы, бичи, :, (, все, идут, к, бшкам, .]"
59875,416428109252935681,@menschenfracht у тебя был 38000 твит и ты не посвятил его мне((((9,-1,"[@, menschenfracht, у, тебя, был, 38000, твит, и, ты, не, посвятил, его, мне, (, (, (, (, 9]"
81948,410761583917559809,"Отучил поляка от лола, установил ему доту и вот он сидит с ботами играет, учится) красота",1,"[отучил, поляка, от, лола, ,, установил, ему, доту, и, вот, он, сидит, с, ботами, играет, ,, учится, ), красота]"
30302,412849571719434240,"что-то мне не нравится мое состояние. это что же, я зря пила дорогие таблетки для профилактики всего пару дней назад?(",-1,"[что-то, мне, не, нравится, мое, состояние, ., это, что, же, ,, я, зря, пила, дорогие, таблетки, для, профилактики, всего, пару, дней, назад, ?, (]"
82707,419853780436676608,"@kostarikara а главное, эти мужики даже не представляют, что ты существуешь и любишь их((",-1,"[@, kostarikara, а, главное, ,, эти, мужики, даже, не, представляют, ,, что, ты, существуешь, и, любишь, их, (, (]"
850,408916299533000704,"RT @Chugunovasweta: Больше некуда писать, всё кавно в прорубь\nПхаха) фантазии нет предела)",1,"[rt, @, chugunovasweta, :, больше, некуда, писать, ,, всё, кавно, в, прорубь, пхаха, ), фантазии, нет, предела, )]"


In [11]:
# data[ data['ttext_'].str.len()<1 ]
max_len = data['ttext_'].str.len().max() # максимальная длинна последовательности
max_len

131

In [12]:
data[ data['ttext_'].str.len()<1 ] # пустые последовательности

,id,ttext,ttype,ttext_


In [13]:
print( 'negative:', len(data.query('ttype==-1')), '\npositive:',len(data.query('ttype==1')) )

negative: 111923 
positive: 114911


In [14]:
data = data[ data['ttext_'].str.len()>0 ].reset_index(drop=True)
print( 'negative:', len(data.query('ttype==-1')), '\npositive:',len(data.query('ttype==1')) )

negative: 111923 
positive: 114911


### кодируем слова

In [15]:
words = set([ w for s in data['ttext_'].apply(set) for w in s ])
vocab = { w:n for n,w in enumerate(sorted(words)) }
print(len(vocab))

315438


In [17]:
# vocab

In [18]:
X  = data['ttext_'].progress_apply(lambda s: [vocab[w] for w in s])

  0%|          | 0/226834 [00:00<?, ?it/s]

In [20]:
X[:3]

0    [38073, 64045, 302732, 313137, 179990, 309098, 119, 218585, 238289, 119, 290997, 212909, 286993, 170187, 266114, 37982, 55261, 222518, 255217, 248750, 286690, 8]
1                                    [159297, 119, 149621, 224915, 216041, 247308, 209825, 214925, 1488, 218585, 206958, 201826, 149603, 257652, 199733, 37982, 55261]
2                                                                    [105584, 38073, 76211, 37982, 219757, 290311, 180510, 8, 313137, 183490, 171855, 285272, 0, 0, 0]
Name: ttext_, dtype: object

---

https://ruscorpora.ru/new/

https://nlpub.ru/Russian_Distributional_Thesaurus    

In [ ]:
import numpy as np
from gensim.models.word2vec import KeyedVectors
# загружаем матрицу word2vec
w2v = KeyedVectors.load_word2vec_format('w2v/all.norm-sz100-w10-cb0-it1-min100.w2v',binary=True,limit=100000)

In [ ]:
# 'радость' in w2v # True
# w2v.get_vector('радость').shape

In [ ]:
# слова нет в словаре
# 'школота' in w2v # False
# w2v.get_vector('школота') # Error!

In [ ]:
X = [ 
 [ w2v.get_vector(w) for w in l if w in w2v ]
 for l in tqdm( data['ttext_'] ) 
]

target = data['ttype']

In [ ]:
w2v_len = len(X[0][0]) # размер представления W2V
w2v_len

In [ ]:
# дополняем последовательности нулями до единого формата,
# выкидываем те, что не нашли соответствия в словаре W2V
z = np.zeros(w2v_len) 
target = np.array([ target[i] for i,xi in enumerate(X) if len(xi)>0 ])
X = np.array([ [z]*(max_len-len(xi)) + xi for xi in X if len(xi)>0  ],dtype=np.float32)
X.shape, target.shape

In [ ]:
del w2v

In [ ]:
target = (target+1)//2
y = np.eye(max(target)+1,dtype=np.float32)[target] # строим выход нейросети (из 2 нейронов)

In [ ]:
from sklearn.model_selection import train_test_split

# делим данные для оценки
x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=.4 )
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
del X
del target
del y

## модель

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.__version__

In [ ]:
input_dim = w2v_len
hidden_dim = 128
n_classes = 2

class Model(nn.Module):
    def __init__(self, 
                 input_dim=input_dim,  # размер входа
                 hid_dim=hidden_dim,   # размер рекуррентного слоя
                 output_dim=n_classes, # размер выхода
                ):
        super().__init__()
        self._rnn = nn.LSTM( input_dim, hid_dim, batch_first=True) # рекуррентный слой
        self._linear = nn.Linear(hid_dim, output_dim) # выходной (линейный) слой

    def forward(self,x):
        _, (hidden, _) = self._rnn(x) # обрабатываем входную последовательность
        # print(hidden.shape,out.shape)
        hidden = hidden.squeeze(0) # прекомпоновка тензора, сокращаем размерность
        out = self._linear(hidden) # считаем выход сети
        out = F.softmax(out,dim=1)
        return out 

In [ ]:
model = Model()
model

In [ ]:
# o =  model.forward( torch.Tensor(x_train[:256]) ) # .to(device) )
# o.shape

### обучаем модель

In [ ]:
# проверяем GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = model.to(device)

In [ ]:
criterion =  nn.CrossEntropyLoss() # ф-ция потери
optimizer = torch.optim.Adam(model.parameters(), lr=.005) # метод оптимизации

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader # генератор батчей

# пакуем данные в формат Torch
dataset_train = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train) )

In [ ]:
from sklearn.metrics import accuracy_score

def accuracy(dataset):
    with torch.set_grad_enabled(False):
        return np.mean([
            accuracy_score(
                y[:,1],
                np.argmax( model(x.to(device)).cpu().numpy(),axis=1 )
            )
            for x,y in DataLoader(dataset, batch_size=len(dataset)//100, shuffle=False)
        ])
        
acc_history = [ accuracy(dataset_train) ] # начальное значение погрешности

In [ ]:
%%time

from torch.utils.data import DataLoader # генератор батчей

loss_history = [ ] # значения ф-ции потери
n_epoch = 7 # количество эпох обучения

for i in tqdm(range(n_epoch)): 
    
    # получаем батч учебных примеров
    for x,y in DataLoader(dataset_train, batch_size=len(y_train)//100, shuffle=True): 
        out = model.forward(x.to(device)) # считаем выход модели
        loss = criterion( y.to(device),out ) # вычисляем значение ф-ции потери
        loss_history.append(loss.item()) # дополняем историю изменения значений ф-ции потери
        optimizer.zero_grad() # очищаем предыдущее значение градиента
        loss.backward()  # вычисляем текущее значение градиента ф-ции потери
        optimizer.step() # корректируем параметры модели
        
    acc_history.append( accuracy(dataset_train) ) #значение погрешности

In [ ]:
from matplotlib import pyplot as plt
# история изменения значений ф-ции потери
plt.plot(loss_history,label='min loss=%.3f'%(min(loss_history)))
plt.grid()
plt.legend()

In [ ]:
# история изменения значений погрешности модели
plt.plot(acc_history,label='max acc=%.3f'%(max(acc_history)),c='r')
plt.grid()
plt.legend()

## проверяем результат

In [ ]:
dataset_test = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test) )

with torch.set_grad_enabled(False):
    o_test = np.hstack([
            np.argmax( model(x.to(device)).cpu().numpy(),axis=1 )
            for x,_ in DataLoader(dataset_test, batch_size=len(dataset_test)//100, shuffle=False)
        ])
    
from sklearn.metrics import classification_report
print(classification_report( np.argmax(y_test,axis=1),o_test))